### project - experimental stuff - hs

In [2]:
# importing libraries etc.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

<hr style="border: 5px solid #ccc">

#### I. filtering data before further cleaning/eda
there are 1115 stores in total. <i>(checked by len(rossmann_store['Store'].unique()) )</i>
that's too much for us to deal with. following instructions from the project description pdf:
<blockquote>2. Cleaning	and	wrangling	data,	i.e.,	taking	top	20%	of	stores	in	terms	of	sales	and	putting	that	into	a	proper	format	</blockquote>

so ig we only need consider the top 20% stores w the best sales, do eda to see what affects their sales, build model, etc etc.
<br>note: we probably won't use `Rossmann_test.csv` cuz it doesn't have sales data so we can't do supervised learning. idk

in this section, basically:
- importing datasets
- limiting the store dataset, `Rossmann_store.csv` we will use to just the top 20% of stores in terms of sales
    - done by: calc avg sales for each store, then take top 20% of stores in terms of sales
- similarly, we wanna limit the other dataset we will use, `Rossmann_train.csv` so that it only gives us details of the stores we want.
    - done by: filtering by the Store IDs based on those present in our t20 store df obtained earlier
- converted `Date` dtype to datetime

unconfirmed but to do:
- merge store dataset (t20 ver.) and sales history (train) dataset? dunno if necessary or not
    - note: set `Date` as index later on, regardless of merge or not
- decide on imputation method & do it

<hr>

In [3]:
# importing datasets
rossmann_store = pd.read_csv("Rossmann_store.csv")
rossmann_sales = pd.read_csv("Rossmann_train.csv") # from here on the train dataset will be referred to as the sales dataset

In [10]:
# creating a function to calculate the average sales for a store
def calc_avg_sales(store_id: int)-> float:
    """This function takes a store ID and returns the average sales value for that store."""
    a_store = rossmann_sales.loc[ (rossmann_sales['Store'] == store_id) ]
    avg_sales = a_store['Sales'].mean()
    return avg_sales.round(2)

# calculating and recording the average sales for each store
store_avg_sales = []
for i in range(1,1116):
    store_avg_sales.append(calc_avg_sales(i))

# adding the average sales data for each store into the store dataset
rossmann_store_w_avg = pd.concat([rossmann_store, pd.DataFrame({'AverageSales' : store_avg_sales})], axis=1)

# taking the top 20 percent of stores in terms of sales
rossmann_store_t20 = \
rossmann_store_w_avg.loc[ (rossmann_store_w_avg['AverageSales'] >= rossmann_store_w_avg['AverageSales'].quantile(0.8)) ]

# filtering the sales dataset
rossmann_sales_t20 = rossmann_sales.loc[ rossmann_sales['Store'].isin(rossmann_store_t20['Store']) ]

In [12]:
# converting date dtype to datetime
pd.to_datetime(rossmann_sales_t20['Date'])

3         2015-07-31
6         2015-07-31
23        2015-07-31
24        2015-07-31
26        2015-07-31
             ...    
1017190   2013-01-01
1017192   2013-01-01
1017194   2013-01-01
1017205   2013-01-01
1017207   2013-01-01
Name: Date, Length: 205466, dtype: datetime64[ns]

#### viewing what we have now below!!!

In [13]:
rossmann_store_t20

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,AverageSales
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,8021.77
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN,7356.90
23,24,a,c,4590.0,3.0,2000.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct",7763.77
24,25,c,a,430.0,4.0,2003.0,0,NaN,NaN,NaN,8892.31
26,27,a,a,60.0,1.0,2005.0,1,5.0,2011.0,"Jan,Apr,Jul,Oct",7838.44
...,...,...,...,...,...,...,...,...,...,...,...
1096,1097,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN,9744.60
1098,1099,a,c,200.0,4.0,2013.0,1,14.0,2013.0,"Jan,Apr,Jul,Oct",7340.34
1100,1101,d,c,4060.0,9.0,2012.0,0,NaN,NaN,NaN,8299.64
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN,8465.28


In [16]:
rossmann_sales_t20

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
3,4,5,2015-07-31,13995,1498,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
23,24,5,2015-07-31,14190,1082,1,1,0,1
24,25,5,2015-07-31,14180,1586,1,1,0,1
26,27,5,2015-07-31,13213,1263,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017190,1097,2,2013-01-01,5961,1405,1,0,a,1
1017192,1099,2,2013-01-01,0,0,0,0,a,1
1017194,1101,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1


- e.g. viewing the sales data for a particular store:

In [15]:
rossmann_sales_t20.loc[rossmann_sales_t20['Store'] == 4]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
3,4,5,2015-07-31,13995,1498,1,1,0,1
1118,4,4,2015-07-30,10387,1276,1,1,0,1
2233,4,3,2015-07-29,10514,1258,1,1,0,1
3348,4,2,2015-07-28,10275,1191,1,1,0,1
4463,4,1,2015-07-27,11812,1379,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1011638,4,6,2013-01-05,10338,1514,1,0,0,0
1012753,4,5,2013-01-04,8290,1232,1,0,0,1
1013868,4,4,2013-01-03,8247,1248,1,0,0,1
1014983,4,3,2013-01-02,9941,1429,1,0,0,1


<hr>

#### for future reference: the missing values rn

In [18]:
rossmann_store_t20.isna().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth     63
CompetitionOpenSinceYear      63
Promo2                         0
Promo2SinceWeek              138
Promo2SinceYear              138
PromoInterval                138
AverageSales                   0
dtype: int64

In [17]:
rossmann_sales_t20.isna().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64